# Object Detection - YOLOv5 (PyTorch)

## 1. Dependencies

This notebook has been tested with **Python 3.8.15** and the following package versions:

In [ ]:
%%capture
!pip install GitPython==3.1.29
!pip install ipython==7.9.0
!pip install verta==0.21.1

## 2. Imports

In [ ]:
import cloudpickle
import concurrent.futures
import json
import os
import pandas as pd
import time
import torch

from verta import Client
from verta.endpoint.autoscaling import Autoscaling
from verta.endpoint.autoscaling.metrics import CpuUtilizationTarget, MemoryUtilizationTarget, RequestsPerWorkerTarget
from verta.endpoint.resources import Resources
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io

## 3. Verta Set Up

In [ ]:
# Use local env vars or uncomment and fill out the lines below:
# os.environ['VERTA_EMAIL'] = ''
# os.environ['VERTA_DEV_KEY'] = ''
# os.environ['VERTA_HOST'] = ''

In [ ]:
client = Client()

In [ ]:
MODEL_NAME = 'Object Detection - PyTorch (Example)'
VERSION = 'v0'
ENDPOINT_NAME = 'object-detection-pytorch'

## 4. Model Class

In [ ]:
class Predictor(VertaModelBase):
    def __init__(self, artifacts=None):
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, trust_repo=True)
    
    @verify_io
    def predict(self, urls):
        result = self.model(urls)
        file_names, img_data = result.files, result.pandas().xyxy
        img_data = [boxes_df.to_dict('records') for boxes_df in img_data]

        return list(zip(file_names, img_data))

    def describe(self):
        return {
            'method': 'predict',
            'args': f"{self.example()}",
            'returns': '[0]',
            'description': 'Identifies objects present in the images.',
            'input_description': 'List of image URLs.',
            'output_description': 'List with identified objects information.'
        }
        
    def example(self):
        return [
            'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_0.jpg',
            'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_1.jpg'
        ]

## 5. Model Test

In [ ]:
model = Predictor()

In [ ]:
results = model.predict(model.example())

In [ ]:
df_results = []

for item in results:
    file_name, img_data = item[0], item[1]
    df = pd.DataFrame.from_dict(img_data)
    df['file'] = file_name
    df_results.append(df)

In [ ]:
pd.DataFrame(pd.concat(df_results)).head()

## 6. Model Register

In [ ]:
registered_model = client.get_or_create_registered_model(name=MODEL_NAME)

In [ ]:
model = registered_model.create_standard_model(
    name = VERSION,
    model_cls = Predictor,
    environment = Python(requirements=[
        "Pillow == 9.3.0",
        "cloudpickle == 2.2.0",
        "ipython == 8.7.0",
        "matplotlib == 3.6.2",
        "opencv-python-headless == 4.6.0.66",
        "psutil == 5.9.4",
        "seaborn == 0.12.1",
        "torch == 1.13.0",
        "torchvision == 0.14.0",
        "transformers == 4.25.1",
        "verta == 0.21.1",
    ])
)

## 7. Auto Scaling and Resources

In [ ]:
autoscaling = Autoscaling(min_replicas=1, max_replicas=20, min_scale=0.1, max_scale=10)
autoscaling.add_metric(CpuUtilizationTarget(0.6))
autoscaling.add_metric(MemoryUtilizationTarget(0.7))
autoscaling.add_metric(RequestsPerWorkerTarget(1))
resources = Resources(cpu=2., memory='12Gi')     

## 8. Model Endpoint

In [ ]:
endpoint = client.get_or_create_endpoint(ENDPOINT_NAME)

In [ ]:
torch_cache_env = {'TORCH_HOME': '/tmp'}

In [ ]:
%%time
endpoint.update(
    model,
    autoscaling = autoscaling,
    resources = resources,
    env_vars = torch_cache_env,
    wait = True
)

In [ ]:
deployed_model = endpoint.get_deployed_model()

## 9. Predictions

In [ ]:
def process_image(url):
    return deployed_model.predict([url])

In [ ]:
urls = [
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_0.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_1.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_2.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_3.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_4.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_5.jpg'
]

In [ ]:
results = []
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    for result in executor.map(process_image, urls):
        results.append(result)
        
end_time = time.time()

In [ ]:
total_time = end_time - start_time
total_time = time.strftime('%Hh %Mm %Ss', time.gmtime(total_time))

In [ ]:
print(f"Processing Time: {total_time} for {len(urls)} URLs.")

In [ ]:
df_results = []

for item in results:
    file_name, img_data = item[0][0], item[0][1]
    df = pd.DataFrame.from_dict(img_data)
    df['file'] = file_name
    df_results.append(df)

In [ ]:
df_result = pd.concat(df_results)

In [ ]:
df_result

In [ ]:
endpoint.delete()